# Application Pipeline

In [1]:
# Imports

import os
import sys
import torch
import ipywidgets as wg
from IPython.display import display, Javascript 
from argparse import ArgumentParser, Namespace

os.sys.path.append(os.path.dirname(os.path.abspath('.')))
from utils.jupyter_widgets import get_pipelin_widget, get_apply_parameter_widgets, get_execution_widgets

In [2]:
# Check for GPU
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('The following GPU was found:\n')
    print('CUDNN VERSION:', torch.backends.cudnn.version())
    print('Number CUDA Devices:', torch.cuda.device_count())
    print('CUDA Device Name:',torch.cuda.get_device_name(0))
    print('CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
else:
    print('No GPU was found. CPU will be used.')

# Select a pipeline 
pipeline = get_pipelin_widget()
display(pipeline)

The following GPU was found:

CUDNN VERSION: 8200
Number CUDA Devices: 1
CUDA Device Name: NVIDIA RTX A4000
CUDA Device Total Memory [GB]: 16.890134528


Dropdown(description='Select Pipeline:', options=('DiffusionModel3d', 'DiffusionModel2d'), style=DescriptionSt…

---
After executing the next block, please adapt all parameters accordingly.
The pipeline expects a list of files that should be used for testing. 
Absolute paths to each files are automatically obtained by concatenating the provided data root and each entry of the file lists.
When changing the selected pipeline, please again execute the following block.

In [3]:
# Define general inference parameters
params = {'output_path': 'results/experiment_name',
          'ckpt_path': 'results/experiment_name/DiffusionModel.ckpt',
          'gpus': use_cuda,
          'overlap': (15,30,30),
          'crop': (15,30,30),
          'input_batch': 'image',
          'clip': (-1.0, 1.0),
          'num_files':-1,
          'add_noise_channel': False,
          'pipeline': pipeline.value,
          }

params_diff = {'timesteps_start': 400,
               'timesteps_save': 100,
               'timesteps_step': 1,
               'blur_sigma': 1
              }


# Load selected pipeline
if  params['pipeline'].lower() == 'diffusionmodel3d':
    from models.DiffusionModel3D import DiffusionModel3D as network
    params.update(params_diff)
elif params['pipeline'].lower() == 'diffusionmodel2d':
    from models.DiffusionModel2D import DiffusionModel2D as network
    params.update(params_diff)
else:
    raise ValueError('Pipeline {0} unknown.'.format(params['pipeline']))

    
# Get and show corresponding parameters
pipeline_args = ArgumentParser(add_help=False)
pipeline_args = network.add_model_specific_args(pipeline_args)
pipeline_args = vars(pipeline_args.parse_known_args()[0])
params = {**params, **pipeline_args}

print('-'*60+'\nPARAMETER FOR PIPELINE "{0}"\n'.format(pipeline.value)+'-'*60)
param_names, widget_list = get_apply_parameter_widgets(params)
for widget in widget_list: 
    display(widget)
    
print('-'*60+'\nEXECUTION SETTINGS\n'+'-'*60)
wg_execute, wg_arguments = get_execution_widgets()
display(wg_arguments)
display(wg_execute)

____________________________________________________________
PARAMETER FOR PIPELINE "DiffusionModel3d"
------------------------------------------------------------


Text(value='results/experiment_name', description='Output Path:', style=DescriptionStyle(description_width='in…

Text(value='results/experiment_name/DiffusionModel.ckpt', description='Checkpoint Path:', style=DescriptionSty…

IntSlider(value=1, description='Use GPU:', max=1, style=SliderStyle(description_width='initial'))

Text(value='15 30 30', description='Overlap (z,y,x):', style=DescriptionStyle(description_width='initial'))

...

BoundedIntText(value=1000, description='Total Number Of Training Timesteps:', max=1000, style=DescriptionStyle…

Dropdown(description='Diffusion Schedule:', layout=Layout(width='max-content'), options=('cosine', 'linear', '…

____________________________________________________________
EXECUTION SETTINGS
------------------------------------------------------------


Checkbox(value=True, description='Get Command Line Arguments', style=DescriptionStyle(description_width='initi…

Checkbox(value=True, description='Execute Now!', style=DescriptionStyle(description_width='initial'))

---
Finish preparations and start processing by executing the next block.

In [ ]:
# Get parameters
param_names = [p for p,w in zip(param_names, widget_list) if w.value!=False and w.value!='']
widget_list = [w for w in widget_list if w.value!=False and w.value!='']
command_line_args = ' '.join(['--pipeline {0}'.format(pipeline.value)]+[n+' '+str(w.value) if not type(w.value)==bool else n for n,w in zip(param_names, widget_list)])

# Show the command line arguments
if wg_arguments.value:
    print('_'*60+'\nCOMMAND LINE ARGUMENTS FOR PIPELINE "{0}"\n'.format(pipeline.value)+'-'*60)
    print(command_line_args)
    print('\n')
        
# Execute the pipeline
if wg_execute.value:
    print('_'*60+'\nEXECUTING PIPELINE "{0}"\n'.format(pipeline.value)+'-'*60)
    %run "../apply_script_diffusion.py" {command_line_args}